In [1]:
!pip install fastai==2.7.9
!pip install fastkaggle==0.0.7
!pip install fasterai==0.1.11
!pip install ipython-autotime==0.3.1
!pip install torch==1.12.1
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.5/225.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 699.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 56.8 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0
  Attempting uninstall: fastai
    Found existing installation: fastai 2.7.12
    Uninstalling fastai-2.7.12:
      Successfully uninstalled fastai-2.7.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.6.0 requires torch==2.0.

In [2]:
from fastkaggle import *
from fastai.vision.all import *
from fasterai.sparse.all import *
from torchvision.models import vgg16_bn

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


time: 6.02 s (started: 2023-05-28 05:45:10 +00:00)


In [3]:
comp = 'paddy-diseases/paddy-diseases'

path = setup_comp(comp, install='"fastcore>=1.4.5" "fastai>=2.7.1" "timm>=0.6.2.dev0"')

time: 13.8 s (started: 2023-05-28 05:45:16 +00:00)


In [4]:
dls = ImageDataLoaders.from_folder(path/"train", 
                                   valid_pct=0.25, 
                                   item_tfms=Resize(256), 
                                   batch_tfms=aug_transforms(   size=64, 
                                                                min_scale=0.75,
                                                                max_rotate=10., 
                                                                min_zoom=0.8, 
                                                                max_zoom=1.2, 
                                                            
                                                            ), 
                                   bs=32)

time: 15.4 s (started: 2023-05-28 05:45:30 +00:00)


In [5]:
def run_once(func):
    """
    A decorator that runs a function only once and returns the result for all subsequent calls.
    """
    memo = {}
    def wrapper(*args, **kwargs):
        if func not in memo:
            memo[func] = func(*args, **kwargs)
        return memo[func]
    return wrapper


time: 656 µs (started: 2023-05-28 05:45:46 +00:00)


In [6]:
@run_once
def get_model_file_size(model): 
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()

    model_size = param_size 
    print('Model size: {} Byte'.format(model_size))


time: 565 µs (started: 2023-05-28 05:45:46 +00:00)


In [7]:
class MemoryCallback(Callback):
    def before_batch(self):
        get_model_file_size(self.learn.model)

time: 531 µs (started: 2023-05-28 05:45:46 +00:00)


In [8]:
sp_cb=[
#       GradientAccumulation(32), 
      # SparsifyCallback(sparsity=12.5, 
      #                    granularity='filter', 
      #                    context='local', 
      #                    criteria=large_final, 
      #                    schedule=one_cycle
      #                    ),
      MemoryCallback(),
]

time: 686 µs (started: 2023-05-28 05:45:46 +00:00)


In [9]:
model = vgg16_bn(num_classes=10)

time: 2.07 s (started: 2023-05-28 05:45:46 +00:00)


In [10]:
learn = Learner(dls, 
                model, 
                metrics=[accuracy,
                         error_rate,
                        ]).to_fp16()
learn.unfreeze()
learn.recorder.train_metrics = True

time: 7.82 ms (started: 2023-05-28 05:45:49 +00:00)


In [11]:
learn.fit_one_cycle(1, 3e-5, cbs=sp_cb)


epoch,train_loss,train_accuracy,train_error_rate,valid_loss,valid_accuracy,valid_error_rate,time
0,2.174146,0.210897,0.789103,1.953595,0.310577,0.689423,01:27


Model size: 537239848 Byte
time: 1min 27s (started: 2023-05-28 05:45:49 +00:00)
